In [2]:
import pyaudio
import numpy as np
from pydub import AudioSegment
from threading import Thread, Event
import time

class AudioPlayer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.audio_segment = AudioSegment.from_file(file_path)
        self.volume = 1.0  # Default volume (1.0 is 100%)
        self.chunk_size = 1024  # 1KB chunks for playback
        self.stream = None
        self.p = pyaudio.PyAudio()
        self.stop_event = Event()
        self.play_thread = Thread(target=self._play_loop)
        self.play_thread.start()

    def _play_loop(self):
        audio_data = np.array(self.audio_segment.get_array_of_samples())
        sample_width = self.audio_segment.sample_width
        channels = self.audio_segment.channels
        rate = self.audio_segment.frame_rate
        audio_format = self.p.get_format_from_width(sample_width)
        num_samples = len(audio_data)

        if self.stream is None:
            self.stream = self.p.open(format=audio_format,
                                      channels=channels,
                                      rate=rate,
                                      output=True)

        idx = 0
        while not self.stop_event.is_set():
            end_idx = min(idx + self.chunk_size, num_samples)
            chunk = audio_data[idx:end_idx]
            chunk = (chunk * self.volume).astype(np.int16)
            self.stream.write(chunk.tobytes())
            idx = end_idx if end_idx < num_samples else 0

    def set_volume(self, volume):
        self.volume = volume

    def stop(self):
        self.stop_event.set()
        if self.stream is not None:
            self.stream.stop_stream()
            self.stream.close()
        self.p.terminate()
        self.play_thread.join()

if __name__ == "__main__":
    file_path = "city_noise.mp3"  # Change this to your audio file path
    player = AudioPlayer(file_path)

    try:
        while True:
            volume = float(input("Enter volume (0.0 to 1.0): "))
            player.set_volume(volume)
            time.sleep(0.1)  # Give a small delay to ensure the volume change takes effect
    except KeyboardInterrupt:
        player.stop()
        print("Audio playback stopped.")


ValueError: could not convert string to float: ''

In [2]:
from ultralytics import YOLO

# Load a YOLOv8n PyTorch model
model = YOLO("be-mine-or-clsm.pt")

# Export the model
model.export(format="openvino", imgsz=256, half=True)  # creates 'yolov8n_openvino_model/'

/Users/jerome/Uni_Master/SonicThinking/sonicthinking/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Ultralytics YOLOv8.2.30 🚀 Python-3.9.6 torch-2.2.2 CPU (Intel Core(TM) i5-4288U 2.60GHz)
YOLOv8m-cls summary (fused): 103 layers, 15767780 parameters, 0 gradients, 41.6 GFLOPs

PyTorch: starting from 'be-mine-or-clsm.pt' with input shape (1, 3, 256, 256) BCHW and output shape(s) (1, 4) (30.2 MB)

OpenVINO: starting export with openvino 2024.1.0-15008-f4afc983258-releases/2024/1...
OpenVINO: export success ✅ 7.0s, saved as 'be-mine-or-clsm_openvino_model/' (60.3 MB)

Export complete (11.6s)
Results saved to /Users/jerome/Uni_Master/SonicThinking
Predict:         yolo predict task=classify model=be-mine-or-clsm_openvino_model imgsz=256  
Validate:        yolo val task=classify model=be-mine-or-clsm_openvino_model imgsz=256 data=/home/jerome/datasets/be-mine-or-1  
Visualize:       https://netron.app


'be-mine-or-clsm_openvino_model'

In [1]:
from openvino.inference_engine import IECore

ie = IECore()
net = ie.read_network(model='be-mine-or-clsm_openvino_model/be-mine-or-clsm.xml', weights='be-mine-or-clsm_openvino_model/be-mine-or-clsm.bin')
input_layer = next(iter(net.input_info))
print("Input layer shape: ", net.input_info[input_layer].input_data.shape)

ModuleNotFoundError: No module named 'openvino.inference_engine'

In [5]:
import openvino.runtime as ov

# Initialize OpenVINO runtime core
core = ov.Core()

# Read the model
model = core.read_model('be-mine-or-clsm_openvino_model/be-mine-or-clsm.xml', 'be-mine-or-clsm_openvino_model/be-mine-or-clsm.bin')

# Get the input information
input_info = model.inputs[0]  # Assuming the model has one input. Adjust if there are multiple inputs.

# Get input name
input_name = input_info.get_any_name()

# Get input shape
input_shape = input_info.get_shape()

# Get input precision
input_precision = input_info.get_element_type()

# Layout is not directly accessible, but you can deduce it from the shape and model context
# For instance, a common input shape for image data is [N, C, H, W], indicating an NCHW layout.

# Print input details
print(f"Input Name: {input_name}")
print(f"Input Shape: {input_shape}")
print(f"Input Precision: {input_precision}")

# You might want to print layout assumptions based on the shape
if len(input_shape) == 4:
    print("Assumed Layout: NCHW or NHWC depending on model specification.")
elif len(input_shape) == 3:
    print("Assumed Layout: CHW or HWC depending on model specification.")
# Add more conditions based on common model input shapes if necessary

Input Name: x
Input Shape: [1,3,256,256]
Input Precision: <Type: 'float32'>
Assumed Layout: NCHW or NHWC depending on model specification.
